# Examining FRIBDAQ Data: The Ion Chamber

Typically, AT-TPC data contains more than just the data produced by the AT-TPC itself. In particular, an upstream ion chamber is critical for selecting the beam of interest entering the AT-TPC. Without this, the data is polluted by reactions involving other beams than the species of interest. This data is typically handled by a separate DAQ called FRIB(NSCL)DAQ.  This notebook will demonstrate the analysis used by Spryal to extract the FRIBDAQ data as well as how it uses this data to improve the AT-TPC results.

First we load the relevant libraries

In [ ]:
import sys
sys.path.append('..')
from spyral.core.config import load_config
from spyral.core.workspace import Workspace
from spyral.core.frib_trace import FRIB_TRACE_LENGTH
from spyral.core.frib_event import FribEvent, IC_COLUMN, SI_COLUMN
from spyral.phase_1 import get_event_range
import h5py as h5
import matplotlib.pyplot as plt
import numpy.random as random
import numpy as np
from pathlib import Path

Now we load our configuration and workspace. While using this notebook one can also customize the configuration on the fly without modifying the acutal JSON file

In [ ]:
config = load_config(Path('../local_config.json'))
# Tweak some parameters
# config.trace.peak_threshold = 1

# Create our workspace
ws = Workspace(config.workspace)

Pick a run and load the raw trace HDF5 file

In [ ]:
run_number = config.run.run_min
trace_file: h5.File = h5.File(ws.get_trace_file_path(run_number))

We select the FRIB group and the evt subgroup (evt is an FRIBDAQ convention meaning the actual event data)

In [ ]:
frib_group: h5.Group = trace_file['frib']
trace_group: h5.Group = frib_group['evt']

Now we select a specific event from the FRIBDAQ data. The event numbers here should match the event numbers in the GET data. By default a random event is selected, but it can be useful to hardcode the event to inspect specific behavior. We then retrieve the traces from the SIS3300 module (id 1903).

In [ ]:
# Ask the trace file for the range of events
min_event, max_event = get_event_range(trace_file)
# Select a random event
event_number = random.randint(min_event, max_event)
print(f'Event {event_number}')
# Can always overwrite with hardcoded event number if needed
# event_number = 4585

trace_data: h5.Dataset = trace_group[f'evt{event_number}_1903']

First lets plot the raw trace for the ion chamber and an auxilary silicon detector

In [ ]:
sample_range = np.arange(0, FRIB_TRACE_LENGTH)
plt.plot(sample_range, trace_data[:, IC_COLUMN])
plt.plot(sample_range, trace_data[:, SI_COLUMN])

Now we'll clean up those traces, removing the baseline, by passing the data to the FribEvent class

In [ ]:
event = FribEvent(trace_data, event_number, config.frib)
plt.plot(sample_range, event.get_ic_trace().trace)
plt.plot(sample_range, event.get_si_trace().trace)

Now we can inspect the peaks identified for the ion chamber

In [ ]:
plt.plot(sample_range, event.get_ic_trace().trace)
for peak in event.get_ic_trace().get_peaks():
    plt.scatter(peak.centroid, peak.amplitude, color='red')
    plt.scatter(peak.negative_inflection, event.get_ic_trace().trace[int(peak.negative_inflection)], color='green')
    plt.scatter(peak.positive_inflection, event.get_ic_trace().trace[int(peak.positive_inflection)], color='green')
plt.show()

As well as the peaks for the silicon

In [ ]:
plt.plot(sample_range, event.get_si_trace().trace)
for peak in event.get_si_trace().get_peaks():
    plt.scatter(peak.centroid, peak.amplitude, color='red')
    plt.scatter(peak.negative_inflection, event.get_si_trace().trace[int(peak.negative_inflection)], color='green')
    plt.scatter(peak.positive_inflection, event.get_si_trace().trace[int(peak.positive_inflection)], color='green')
plt.show()

Finally, we can use the peaks to identify the "good" ion chamber peak. A good ion chamber peak is identified as an ion chamber peak that *does not* have a coincident silicon peak. If the good ion chamber peak is not the first peak in the ion chamber spectrum, this means that the trigger was acutally offset by the wrong beam event. We can correct for this by calculating the time difference between the earliest ion chamber peak and the good ion chamber peak. Additionally, the configuration can controll the maximum allowed multiplicity for the ion chamber. By default the only singles events are allowed.

In [ ]:
good_peak = event.get_good_ic_peak(config.frib)
print(good_peak)
if good_peak is not None:
    ic_offset = event.correct_ic_time(good_peak, config.detector.get_frequency)
    print(f'IC Time Offset in GET Buckets: {ic_offset}')